In [ ]:
import pandas as pd 
from datetime import datetime as dt 
import os
import glob 
from envoke import *
from email_content import html_content 
import time 

In [ ]:
load_dotenv()

In [ ]:
# Selecting oldest file from "Survey Distribution" folder - one to be added each week on Wednesday, script run by Friday 
file_list = glob.glob("files/List/*.xlsx")
latest_file = max(file_list, key=os.path.getctime)
email_list = pd.read_excel(latest_file)
email_list.columns = ["return_number", "row_check", "modified_on", "name", "corp_name", "return_type", "fiscal_year", "cao_revision", "contact_filing_return", 
                      "contact_filing_noc", "created_on", "created_by", "email_contact", "email_registration", "first_name", "last_name"]



email_list.index = email_list.email_registration
email_list_less = email_list.drop_duplicates(subset=["email_registration"])
file_list = glob.glob("files/Logs/*.csv")

email_list = email_list_less.index.to_list()
past_emails = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)["email_contact"].unique().tolist()
to_send = [email for email in email_list if email not in past_emails]
email_list_less = email_list_less[email_list_less["email_contact"].isin(to_send)]

contact_dicts = create_contact_dicts(email_list_less)

In [ ]:
with requests.sessions.Session() as session: # need to account for time sleep and duplicates lol 
    session.auth = (os.getenv("USERNAME"), os.getenv("PASSWORD")) 
    value_list = []

    for contact in contact_dicts:
        

        response = upsert_contact(contact, session.auth)
        time.sleep(0.4)

        if response.json()['success']: 
            response = send_survey(contact, html_content, session.auth)
            time.sleep(0.4)
            print(f"Successful delivery to {contact["email"]}")
            
        else: 
            value_list.append({contact["email"]: response.text})
            print(f"Error processing {contact["email"]}, {response.text}")
    
    if len(value_list) > 0: 
        print(value_list)
    else: 
        print("Complete")




In [ ]:
contact_log = email_list_less.groupby('email_contact').name.count().to_frame()
contact_log["survey_sent_at"] = dt.datetime.now().strftime("%Y-%m-%d")
contact_log["is_contact"] = pd.NA
contact_log["added_on"] = pd.NA

contact_log.to_csv(f"files/Logs/{dt.datetime.now().strftime("%Y-%m-%d")}_SurveyLog.csv")
